In [ ]:
%pip install langchain
%pip install openai

创建一个基于 openai 的大语言模型

In [15]:
from langchain import OpenAI
import openai
llm = OpenAI(
    temperature=0,
	openai_api_key='<OPENAI-API-KEY>',
	model_name="text-davinci-003"
)

定义一个回调用于展示每次调用详情

In [20]:
from langchain.callbacks import get_openai_callback
def track_tokens_usage(llm, query):
    with get_openai_callback() as cb:
        result = llm(query)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Successful Requests: {cb.successful_requests}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    return result

def chain_tokens_usage(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Successful Requests: {cb.successful_requests}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    return result

In [19]:
track_tokens_usage(llm,"如何使用 langchain ")

Total Tokens: 265
Prompt Tokens: 10
Completion Tokens: 255
Successful Requests: 1
Total Cost (USD): $0.0053


'\n\n1. 首先，需要在网站上注册一个账号，并登录。\n\n2. 然后，可以在网站上搜索想要学习的语言，并选择一个课程。\n\n3. 接下来，可以开始学习课程，每个课程都有一系列的课程任务，可以根据自己的需要来完成。\n\n4. 完成课程任务后，可以在网站上查看自己的学习'

以上为一次简单的调用, 并通过回调函数( track_tokens_usage) 获取到一次调用的消耗量, 
这种使用,不具备理解上下文的能力,  
通过 openai接口调用, 如果想实现理解上下文的能力, 就需要在提交新的对话的时候,需要一起送入之前已经提交的问题以及答案
这里面涉及到一下几个类  都是 基于内存的上下文理解 
1. Buffer 相关指代是不进行摘要处理 简单除暴的进行上下文拼接, 
2. Summary 会对上文所有的内容进行摘要处理, 只记录摘要信息
```
ConversationSummaryMemory 会对问答进行摘要处理 由于会请求生产摘要处理所以会请求两次接口
ConversationBufferMemory  记录原始的请求内容 , 不进行摘要处理,  之请求一次内容
ConversationBufferWindowMemory
ConversationSummaryBufferMemory
```

In [21]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import ConversationChain

conversation = ConversationChain(llm=llm, memory = ConversationBufferMemory())


In [22]:
chain_tokens_usage(conversation,"今天天气不错")

Total Tokens: 162
Prompt Tokens: 72
Completion Tokens: 90
Successful Requests: 1
Total Cost (USD): $0.00324


' 是的，今天的天气真是太棒了！我喜欢这种晴朗的天气，它让我感觉很放松。你有什么打算吗？'

打印缓存内容

In [23]:
print(conversation.memory.buffer)

Human: 今天天气不错
AI:  是的，今天的天气真是太棒了！我喜欢这种晴朗的天气，它让我感觉很放松。你有什么打算吗？


In [24]:
chain_tokens_usage(conversation,"你有什么建议吗")

Total Tokens: 298
Prompt Tokens: 185
Completion Tokens: 113
Successful Requests: 1
Total Cost (USD): $0.00596


' 嗯，我建议你可以去散步，或者去公园野餐，或者去海边放松一下。你可以做任何你喜欢的事情，只要你感到开心就好。'

In [25]:
print(conversation.memory.buffer)

Human: 今天天气不错
AI:  是的，今天的天气真是太棒了！我喜欢这种晴朗的天气，它让我感觉很放松。你有什么打算吗？
Human: 你有什么建议吗
AI:  嗯，我建议你可以去散步，或者去公园野餐，或者去海边放松一下。你可以做任何你喜欢的事情，只要你感到开心就好。


In [26]:
conversationSu = ConversationChain(llm=llm, memory = ConversationSummaryMemory(llm=llm))
chain_tokens_usage(conversationSu,"今天天气不错")
print(conversationSu.memory.buffer)

Total Tokens: 455
Prompt Tokens: 327
Completion Tokens: 128
Successful Requests: 2
Total Cost (USD): $0.0091
Total Tokens: 617
Prompt Tokens: 438
Completion Tokens: 179
Successful Requests: 2
Total Cost (USD): $0.01234

The human commented on the nice weather and the AI agreed, saying it was a great day and that it made them feel relaxed. The AI then asked the human what their plans were and suggested going for a walk, visiting a park, or looking at the scenery around them. The human asked if the AI had any other suggestions.


In [28]:
conversationSu1 = ConversationChain(llm=llm, memory = ConversationSummaryMemory(llm=llm))
chain_tokens_usage(conversationSu1,"今天天气不错,返回中文回答")
print(conversationSu1.memory.buffer)

Total Tokens: 427
Prompt Tokens: 328
Completion Tokens: 99
Successful Requests: 2
Total Cost (USD): $0.008539999999999999

The human asked the AI a question in Chinese about the weather. The AI responded that the weather was great and it was a perfect day for a walk.


In [ ]:
chain_tokens_usage(conversationSu1,"你有什么建议吗")
print(conversationSu1.memory.buffer)